Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [46]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [47]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [48]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [49]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [50]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_input = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  biases_input = tf.Variable(tf.zeros([1024]))
  weights_hidden = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  biases_hidden = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits_input = tf.matmul(tf_train_dataset, weights_input) + biases_input
  logits_hidden = tf.matmul(tf.nn.relu(logits_input), weights_hidden) + biases_hidden
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_hidden) +
    0.01*(tf.nn.l2_loss(weights_input) + 
         tf.nn.l2_loss(biases_input) +
         tf.nn.l2_loss(weights_hidden) + 
         tf.nn.l2_loss(biases_hidden)))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_hidden)

  valid_logits = tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_input) + biases_input), weights_hidden) + biases_hidden
  valid_prediction = tf.nn.softmax(valid_logits)

  test_logits = tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_input) + biases_input), weights_hidden) + biases_hidden
  test_prediction = tf.nn.softmax(test_logits)

In [51]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3480.123535
Minibatch accuracy: 9.4%
Validation accuracy: 28.1%
Minibatch loss at step 500: 21.247269
Minibatch accuracy: 84.4%
Validation accuracy: 85.0%
Minibatch loss at step 1000: 0.829877
Minibatch accuracy: 86.7%
Validation accuracy: 83.8%
Minibatch loss at step 1500: 0.756320
Minibatch accuracy: 85.2%
Validation accuracy: 82.8%
Minibatch loss at step 2000: 0.625503
Minibatch accuracy: 85.9%
Validation accuracy: 83.5%
Minibatch loss at step 2500: 0.686097
Minibatch accuracy: 85.9%
Validation accuracy: 83.5%
Minibatch loss at step 3000: 0.704295
Minibatch accuracy: 85.9%
Validation accuracy: 82.1%
Test accuracy: 89.3%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [52]:
num_steps = 3001

train_d = train_dataset[:batch_size*10]
train_l = train_labels[:batch_size*10]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_l.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_d[offset:(offset + batch_size), :]
    batch_labels = train_l[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3471.234863
Minibatch accuracy: 10.9%
Validation accuracy: 35.0%
Minibatch loss at step 500: 21.134684
Minibatch accuracy: 100.0%
Validation accuracy: 79.3%
Minibatch loss at step 1000: 0.573340
Minibatch accuracy: 98.4%
Validation accuracy: 80.2%
Minibatch loss at step 1500: 0.405667
Minibatch accuracy: 100.0%
Validation accuracy: 79.2%
Minibatch loss at step 2000: 0.404277
Minibatch accuracy: 99.2%
Validation accuracy: 80.0%
Minibatch loss at step 2500: 0.387271
Minibatch accuracy: 100.0%
Validation accuracy: 79.6%
Minibatch loss at step 3000: 0.399015
Minibatch accuracy: 98.4%
Validation accuracy: 80.0%
Test accuracy: 87.2%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [103]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_input = tf.Variable(
    tf.truncated_normal([image_size * image_size, 1024]))
  biases_input = tf.Variable(tf.zeros([1024]))
  weights_hidden = tf.Variable(
    tf.truncated_normal([1024, num_labels]))
  biases_hidden = tf.Variable(tf.zeros([num_labels]))
  keep_prob = tf.placeholder(tf.float32)
  
  # Training computation.
  logits_input = tf.matmul(tf_train_dataset, weights_input) + biases_input
  dropout = tf.nn.dropout(tf.nn.relu(logits_input), keep_prob)
  logits_hidden = tf.matmul(dropout, weights_hidden) + biases_hidden
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_hidden) +
    0.01*(tf.nn.l2_loss(weights_input) + 
         tf.nn.l2_loss(biases_input) +
         tf.nn.l2_loss(weights_hidden) + 
         tf.nn.l2_loss(biases_hidden)))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_hidden)

  valid_logits = tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_input) + biases_input), weights_hidden) + biases_hidden
  valid_prediction = tf.nn.softmax(valid_logits)

  test_logits = tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_input) + biases_input), weights_hidden) + biases_hidden
  test_prediction = tf.nn.softmax(test_logits)

In [104]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3581.838379
Minibatch accuracy: 7.0%
Validation accuracy: 36.1%
Minibatch loss at step 500: 21.504780
Minibatch accuracy: 78.1%
Validation accuracy: 83.2%
Minibatch loss at step 1000: 0.907858
Minibatch accuracy: 85.2%
Validation accuracy: 83.3%
Minibatch loss at step 1500: 0.835212
Minibatch accuracy: 85.2%
Validation accuracy: 82.8%
Minibatch loss at step 2000: 0.671692
Minibatch accuracy: 86.7%
Validation accuracy: 82.6%
Minibatch loss at step 2500: 0.754056
Minibatch accuracy: 84.4%
Validation accuracy: 83.0%
Minibatch loss at step 3000: 0.746647
Minibatch accuracy: 84.4%
Validation accuracy: 81.1%
Test accuracy: 88.3%


In [55]:
num_steps = 3001

train_d = train_dataset[:batch_size*10]
train_l = train_labels[:batch_size*10]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_l.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_d[offset:(offset + batch_size), :]
    batch_labels = train_l[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3631.361084
Minibatch accuracy: 10.2%
Validation accuracy: 25.6%
Minibatch loss at step 500: 21.281334
Minibatch accuracy: 98.4%
Validation accuracy: 79.9%
Minibatch loss at step 1000: 0.602618
Minibatch accuracy: 96.9%
Validation accuracy: 80.1%
Minibatch loss at step 1500: 0.442070
Minibatch accuracy: 99.2%
Validation accuracy: 79.7%
Minibatch loss at step 2000: 0.430290
Minibatch accuracy: 99.2%
Validation accuracy: 80.2%
Minibatch loss at step 2500: 0.425731
Minibatch accuracy: 98.4%
Validation accuracy: 79.8%
Minibatch loss at step 3000: 0.430214
Minibatch accuracy: 97.7%
Validation accuracy: 79.7%
Test accuracy: 87.1%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [56]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_input = tf.Variable(
    tf.truncated_normal([image_size * image_size, 2048]))
  biases_input = tf.Variable(tf.zeros([2048]))
  weights_hidden = tf.Variable(
    tf.truncated_normal([2048, num_labels]))
  biases_hidden = tf.Variable(tf.zeros([num_labels]))
  keep_prob = tf.placeholder(tf.float32)
  global_step = tf.Variable(0)  # count the number of steps taken.
  
  # Training computation.
  logits_input = tf.matmul(tf_train_dataset, weights_input) + biases_input
  dropout = tf.nn.dropout(tf.nn.relu(logits_input), keep_prob)
  logits_hidden = tf.matmul(dropout, weights_hidden) + biases_hidden
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits_hidden) +
    0.01*(tf.nn.l2_loss(weights_input) + 
         tf.nn.l2_loss(biases_input) +
         tf.nn.l2_loss(weights_hidden) + 
         tf.nn.l2_loss(biases_hidden)))
  
  # Optimizer.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 100000, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits_hidden)

  valid_logits = tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_input) + biases_input), weights_hidden) + biases_hidden
  valid_prediction = tf.nn.softmax(valid_logits)

  test_logits = tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_input) + biases_input), weights_hidden) + biases_hidden
  test_prediction = tf.nn.softmax(test_logits)

In [57]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 6999.883789
Minibatch accuracy: 10.9%
Validation accuracy: 28.6%
Minibatch loss at step 500: 42.792480
Minibatch accuracy: 82.0%
Validation accuracy: 84.4%
Minibatch loss at step 1000: 1.038132
Minibatch accuracy: 86.7%
Validation accuracy: 83.3%
Minibatch loss at step 1500: 0.829125
Minibatch accuracy: 81.2%
Validation accuracy: 80.9%
Minibatch loss at step 2000: 0.654498
Minibatch accuracy: 88.3%
Validation accuracy: 82.8%
Minibatch loss at step 2500: 0.735166
Minibatch accuracy: 84.4%
Validation accuracy: 82.6%
Minibatch loss at step 3000: 0.754756
Minibatch accuracy: 85.2%
Validation accuracy: 81.7%
Test accuracy: 88.8%


In [129]:
batch_size = 128
learning_rate = 0.001
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons

graph = tf.Graph()
with graph.as_default():
  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  keep_prob = tf.placeholder(tf.float32)
  
  # Variables.
  initializer = tf.contrib.layers.xavier_initializer()
  weights = {
    'h1': tf.Variable(initializer([image_size * image_size, n_hidden_1])),
    'h2': tf.Variable(initializer([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(initializer([n_hidden_2, num_labels]))
  }
  biases = {
    'b1': tf.Variable(initializer([n_hidden_1])),
    'b2': tf.Variable(initializer([n_hidden_2])),
    'out': tf.Variable(initializer([num_labels]))
  }
  keep_prob = tf.placeholder(tf.float32)
  global_step = tf.Variable(0)  # count the number of steps taken.
  
  # Create model
  def multilayer_perceptron(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.nn.relu(tf.add(tf.matmul(x, weights['h1']), biases['b1']))
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.nn.relu(tf.add(tf.matmul(layer_1, weights['h2']), biases['b2']))
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

  # Construct model
  logits = multilayer_perceptron(tf_train_dataset)
  train_prediction = tf.nn.softmax(logits)
    
  valid_logits = multilayer_perceptron(tf_valid_dataset)
  valid_prediction = tf.nn.softmax(valid_logits)

  test_logits = multilayer_perceptron(tf_test_dataset)
  test_prediction = tf.nn.softmax(test_logits)

  # Define loss and optimizer
  loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=tf_train_labels)+
    0.01*(tf.nn.l2_loss(weights['h1']) + 
         tf.nn.l2_loss(biases['b1']) +
         tf.nn.l2_loss(weights['h2']) + 
         tf.nn.l2_loss(biases['b2']) + 
         tf.nn.l2_loss(weights['out']) + 
         tf.nn.l2_loss(biases['out'])))

  learning_rate = tf.train.exponential_decay(0.1, global_step, 100000, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

In [130]:
num_steps = 5001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels, keep_prob: 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 5.688591
Minibatch accuracy: 7.8%
Validation accuracy: 16.2%
Minibatch loss at step 500: 1.889847
Minibatch accuracy: 83.6%
Validation accuracy: 84.0%
Minibatch loss at step 1000: 1.130543
Minibatch accuracy: 87.5%
Validation accuracy: 84.5%
Minibatch loss at step 1500: 0.901180
Minibatch accuracy: 84.4%
Validation accuracy: 84.4%
Minibatch loss at step 2000: 0.683382
Minibatch accuracy: 90.6%
Validation accuracy: 84.6%
Minibatch loss at step 2500: 0.725281
Minibatch accuracy: 86.7%
Validation accuracy: 84.8%
Minibatch loss at step 3000: 0.715587
Minibatch accuracy: 85.2%
Validation accuracy: 84.4%
Minibatch loss at step 3500: 0.707498
Minibatch accuracy: 87.5%
Validation accuracy: 85.0%
Minibatch loss at step 4000: 0.737755
Minibatch accuracy: 83.6%
Validation accuracy: 84.8%
Minibatch loss at step 4500: 0.666613
Minibatch accuracy: 86.7%
Validation accuracy: 84.7%
Minibatch loss at step 5000: 0.646557
Minibatch accuracy: 88.3%
Validation accuracy